# NLP and Neural Networks

In this exercise, we'll apply our knowledge of neural networks to process natural language. As we did in the bigram exercise, the goal of this lab is to predict the next word, given the previous one.

### Data set

Load the text from "One Hundred Years of Solitude" that we used in our bigrams exercise. It's located in the data folder.

### Important note:

Start with a smaller part of the text. Maybe the first 10 parragraphs, as the number of tokens rapidly increases as we add more text. 

Later you can use a bigger corpus.

In [35]:
import re
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
from collections import Counter


# Paso 1: Cargar el archivo de texto
file_path = 'data\cap1.txt'

with open(file_path, 'r', encoding='utf-8') as file:
    text = file.read()

# Paso 2: Preprocesar el texto
text = text.lower() # Convertir el texto a minúsculas
text = re.sub(r'\s+', ' ', text)  # Eliminar espacios extra



Don't forget to prepare the data by generating the corresponding tokens.

In [36]:
from nltk.tokenize import word_tokenize
# Tokenizar el texto
tokens = word_tokenize(text)

# Mostrar los primeros 10 tokens
print(tokens[:10])


['muchos', 'años', 'después', ',', 'frente', 'al', 'pelotón', 'de', 'fusilamiento', ',']


### Let's prepare the data set.

Our neural network needs to have an input X and an output y. Remember that these sets are numerical, so you'd need something to map the tokens into numbers, and viceversa.

In [37]:
# in this case, let's consider a bigram (w1, w2)
# assign the w1 to the X vector, and w2 to the y vector, why do we do this?
'''Asignamos w1 a X y w2 a y, para entrenar el modelo y prediga la próxima palabra 
dada la palabra actual, capturando así las relaciones directas entre palabras'''

# Crear diccionario de bigramas
bigrams = {}
for w1, w2 in zip(tokens, tokens[1:]):
    bigram = (w1, w2)
    bigrams[bigram] = bigrams.get(bigram, 0) + 1

# Mostrar algunos bigramas y sus conteos
print(list(bigrams.items())[:10])



[(('muchos', 'años'), 3), (('años', 'después'), 2), (('después', ','), 2), ((',', 'frente'), 1), (('frente', 'al'), 1), (('al', 'pelotón'), 2), (('pelotón', 'de'), 2), (('de', 'fusilamiento'), 2), (('fusilamiento', ','), 2), ((',', 'el'), 10)]


In [38]:
# Don't forget that since we are using torch, our training set vectors should be tensors
# Crear el vocabulario a partir de tokens
word_to_idx = {word: idx + 1 for idx, word in enumerate(set(tokens))}
idx_to_word = {idx: word for word, idx in word_to_idx.items()}

# Preparar las secuencias de entrada y salida
input_sequences = []
output_sequences = []

for (w1, w2), count in bigrams.items():
    input_sequences.append(word_to_idx[w1])
    output_sequences.append(word_to_idx[w2])

# Convertir listas a tensores
input_tensor = torch.tensor(input_sequences, dtype=torch.long)
output_tensor = torch.tensor(output_sequences, dtype=torch.long)



In [39]:
# Note that our vectors are integers, which can be thought as a categorical variables.
# torch provides the one_hot method, that would generate tensors suitable for our nn
# make sure that the dtype of your tensor is float.

# Convertir entradas a codificación one-hot
vocab_size = len(word_to_idx) + 1  # Tamaño del vocabulario
input_one_hot = F.one_hot(input_tensor, num_classes=vocab_size).float()

### Network design
To start, we are going to have a very simple network. Define a single layer network

In [55]:
# How many neurons should our input layer have?
# Use as many neurons as the total number of categories (from your one-hot encoded tensors)

# Definir el modelo de red neuronal simple
class SimpleBigramModel(nn.Module):
    def __init__(self, vocab_size):
        super(SimpleBigramModel, self).__init__()
        self.fc = nn.Linear(vocab_size, vocab_size)  # Capa lineal
    
    def forward(self, x):
        out = self.fc(x)
        return torch.softmax(out, dim=1)  # Activación softmax para convertir en probabilidades

# Inicializar el modelo, la función de pérdida y el optimizador
model = SimpleBigramModel(vocab_size)
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.01)

# Mostrar la estructura de la red
print(model)



SimpleBigramModel(
  (fc): Linear(in_features=2033, out_features=2033, bias=True)
)


In [56]:
# Train your network

# Número de épocas para entrenamiento
num_epochs = 50

# Ciclo de entrenamiento
for epoch in range(num_epochs):
    # Resetear gradientes
    optimizer.zero_grad()
    
    # Forward pass
    outputs = model(input_one_hot)
    
    # Calcular la pérdida
    loss = criterion(outputs, output_tensor)
    
    # Backward pass y optimización
    loss.backward()
    optimizer.step()
    
    # Mostrar la pérdida cada 10 épocas
    if (epoch + 1) % 10 == 0:
        print(f'Época [{epoch + 1}/{num_epochs}], Pérdida: {loss.item():.4f}')

print("Entrenamiento completado.")



Época [10/50], Pérdida: 7.6172
Época [20/50], Pérdida: 7.6170
Época [30/50], Pérdida: 7.6169
Época [40/50], Pérdida: 7.6166
Época [50/50], Pérdida: 7.6162
Entrenamiento completado.


### Analysis

1. Test your network with a few words

In [67]:
import torch
import torch.nn as nn
import torch.nn.functional as F

# Seleccionar algunas palabras para probar
test_words = ['macondo', 'soledad', 'años', 'los']

# Convertir palabras de prueba a índices y luego a codificación one-hot
test_indices = [word_to_idx[word] for word in test_words]
test_tensor = torch.tensor(test_indices, dtype=torch.long)
test_one_hot = F.one_hot(test_tensor, num_classes=vocab_size).float()

# Obtener las predicciones de la red
outputs = model(test_one_hot)

# Definir la función de pérdida para calcular la verosimilitud negativa
criterion = nn.CrossEntropyLoss()

# Mostrar las predicciones y calcular la verosimilitud negativa
for i, word in enumerate(test_words):
    print(f"Palabra de prueba: {word}")
    print(f"Predicciones (Top 5):")
    top5_prob, top5_idx = torch.topk(outputs[i], 5)
    
    for j in range(5):
        predicted_word = idx_to_word[top5_idx[j].item()]
        print(f"  {predicted_word}: {top5_prob[j].item():.4f}")
    
    # Calcular la verosimilitud negativa para la palabra
    target_idx = torch.tensor([test_indices[i]], dtype=torch.long)
    negative_likelihood = criterion(outputs[i].unsqueeze(0), target_idx)
    
    print(f"Verosimilitud negativa para '{word}': {negative_likelihood.item():.4f}\n")


Palabra de prueba: macondo
Predicciones (Top 5):
  se: 0.0024
  a: 0.0024
  ,: 0.0024
  en: 0.0024
  para: 0.0024
Verosimilitud negativa para 'macondo': 7.6165

Palabra de prueba: soledad
Predicciones (Top 5):
  y: 0.0023
  un: 0.0014
  ,: 0.0014
  hasta: 0.0014
  le: 0.0014
Verosimilitud negativa para 'soledad': 7.6173

Palabra de prueba: años
Predicciones (Top 5):
  y: 0.0024
  en: 0.0024
  ,: 0.0024
  más: 0.0023
  .: 0.0022
Verosimilitud negativa para 'años': 7.6165

Palabra de prueba: los
Predicciones (Top 5):
  años: 0.0026
  más: 0.0026
  gitanos: 0.0025
  hombres: 0.0024
  niños: 0.0023
Verosimilitud negativa para 'los': 7.6166



In [66]:
import torch
import torch.nn as nn
import torch.nn.functional as F

# Definir una función para generar una frase con el modelo simple
def generate_sentence(start_word, model, length=5):
    current_word = start_word
    sentence = [current_word]
    
    for _ in range(length):
        idx = word_to_idx.get(current_word, 0)  # Obtener el índice de la palabra actual
        input_tensor = torch.tensor([idx], dtype=torch.long)
        input_one_hot = F.one_hot(input_tensor, num_classes=vocab_size).float()
        
        output = model(input_one_hot)  # Obtener la predicción del modelo
        top_word_idx = torch.argmax(output, dim=1).item()  # Índice de la palabra más probable
        current_word = idx_to_word.get(top_word_idx, '<UNK>')  # Obtener la palabra correspondiente
        
        sentence.append(current_word)  # Añadir la palabra a la oración
    
    return ' '.join(sentence)

# Función para calcular la verosimilitud negativa de una frase
def calculate_negative_likelihood(sentence, model):
    test_indices = [word_to_idx[word] for word in sentence]
    test_tensor = torch.tensor(test_indices[:-1], dtype=torch.long)  # Excluir la última palabra
    test_target = torch.tensor(test_indices[1:], dtype=torch.long)   # Excluir la primera palabra
    
    test_one_hot = F.one_hot(test_tensor, num_classes=vocab_size).float()
    outputs = model(test_one_hot)
    
    criterion = nn.CrossEntropyLoss()
    loss = criterion(outputs, test_target)
    
    return loss.item()

# Generar frases con el modelo simple
print(generate_sentence('macondo', model))
print(generate_sentence('soledad', model))
print(generate_sentence('los', model))

# Calcular la verosimilitud negativa de las frases generadas
generated_sentence_1 = generate_sentence('macondo', model).split()
generated_sentence_2 = generate_sentence('soledad', model).split()
generated_sentence_3 = generate_sentence('los', model).split()

print(f"Verosimilitud negativa para la oración '{' '.join(generated_sentence_1)}': {calculate_negative_likelihood(generated_sentence_1, model):.4f}")
print(f"Verosimilitud negativa para la oración '{' '.join(generated_sentence_2)}': {calculate_negative_likelihood(generated_sentence_2, model):.4f}")
print(f"Verosimilitud negativa para la oración '{' '.join(generated_sentence_3)}': {calculate_negative_likelihood(generated_sentence_3, model):.4f}")


macondo se lo que lo que
soledad y las de melquíades se
los años y las de melquíades
Verosimilitud negativa para la oración 'macondo se lo que lo que': 7.6153
Verosimilitud negativa para la oración 'soledad y las de melquíades se': 7.6152
Verosimilitud negativa para la oración 'los años y las de melquíades': 7.6152


**2. What does each value in the tensor represents?**

Cada valor en el tensor de salida representa la probabilidad predicha de que una palabra específica en el vocabulario sea la siguiente palabra, dado el contexto de entrada. Estos valores son generados por la función softmax, que convierte los puntajes en probabilidades que suman a 1.

**3. Why does it make sense to choose that number of neurons in our layer?**

Tiene sentido elegir el número de neuronas en la capa de entrada igual al tamaño del vocabulario porque cada neurona corresponde a una palabra única en el conjunto de datos. Esto permite que el modelo aprenda una representación de cada palabra y pueda hacer predicciones precisas sobre la siguiente palabra en base a la entrada.

**4. What's the negative likelihood for each example?**

Para cada palabra generada el valor esta por 7.61, lo que se considera alto

**5. Try generating a few sentences?**

Se ha realizado

**6. What's the negative likelihood for each sentence?**

Para cada frase generada el valor esta por 7.61, lo que se considera alto

### Design your own neural network (more layers and different number of neurons)
The goal is to get sentences that make more sense 

In [118]:
# Definir el modelo de red neuronal con capas más profundas
class DeepBigramModel(nn.Module):
    def __init__(self, vocab_size, hidden_size=8, dropout_rate=0.5):
        super(DeepBigramModel, self).__init__()
        self.fc1 = nn.Linear(vocab_size, hidden_size)
        self.fc2 = nn.Linear(hidden_size, hidden_size)
        self.fc3 = nn.Linear(hidden_size, vocab_size)
        self.dropout = nn.Dropout(dropout_rate)
    
    def forward(self, x):
        x = F.relu(self.fc1(x))
        x = self.dropout(x)
        x = F.relu(self.fc2(x))
        x = self.dropout(x)
        x = self.fc3(x)
        return torch.softmax(x, dim=1)

# Inicializar el modelo, la función de pérdida y el optimizador
model = DeepBigramModel(vocab_size)
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.0001)

# Número de épocas para entrenamiento
num_epochs = 25

# Ciclo de entrenamiento
for epoch in range(num_epochs):
    # Resetear gradientes
    optimizer.zero_grad()
    
    # Forward pass
    outputs = model(input_one_hot)
    
    # Calcular la pérdida
    loss = criterion(outputs, output_tensor)
    
    # Backward pass y optimización
    loss.backward()
    optimizer.step()
    
    # Mostrar la pérdida cada 10 épocas
    if (epoch + 1) % 10 == 0:
        print(f'Época [{epoch + 1}/{num_epochs}], Pérdida: {loss.item():.4f}')

print("Entrenamiento completado.")


Época [10/25], Pérdida: 7.6173
Época [20/25], Pérdida: 7.6173
Entrenamiento completado.


In [119]:
import torch
import torch.nn as nn
import torch.nn.functional as F

# Seleccionar algunas palabras para probar
test_words = ['macondo', 'soledad', 'años', 'los']

# Convertir palabras de prueba a índices y luego a codificación one-hot
test_indices = [word_to_idx[word] for word in test_words]
test_tensor = torch.tensor(test_indices, dtype=torch.long)
test_one_hot = F.one_hot(test_tensor, num_classes=vocab_size).float()

# Obtener las predicciones de la red
outputs = model(test_one_hot)

# Definir la función de pérdida para calcular la verosimilitud negativa
criterion = nn.CrossEntropyLoss()

# Mostrar las predicciones y calcular la verosimilitud negativa
for i, word in enumerate(test_words):
    print(f"Palabra de prueba: {word}")
    print(f"Predicciones (Top 5):")
    top5_prob, top5_idx = torch.topk(outputs[i], 5)
    
    for j in range(5):
        predicted_word = idx_to_word[top5_idx[j].item()]
        print(f"  {predicted_word}: {top5_prob[j].item():.4f}")
    
    # Calcular la verosimilitud negativa para la palabra
    target_idx = torch.tensor([test_indices[i]], dtype=torch.long)
    negative_likelihood = criterion(outputs[i].unsqueeze(0), target_idx)
    
    print(f"Verosimilitud negativa para '{word}': {negative_likelihood.item():.4f}\n")

Palabra de prueba: macondo
Predicciones (Top 5):
  adiestrarlos: 0.0008
  abejas: 0.0008
  podrá: 0.0008
  al: 0.0008
  arrastrado: 0.0008
Verosimilitud negativa para 'macondo': 7.6171

Palabra de prueba: soledad
Predicciones (Top 5):
  seguir: 0.0007
  internas: 0.0007
  descendencia: 0.0007
  avanzaron: 0.0007
  brújula: 0.0007
Verosimilitud negativa para 'soledad': 7.6173

Palabra de prueba: años
Predicciones (Top 5):
  adiestrarlos: 0.0008
  abejas: 0.0008
  podrá: 0.0008
  arrastrado: 0.0008
  espera: 0.0008
Verosimilitud negativa para 'años': 7.6171

Palabra de prueba: los
Predicciones (Top 5):
  seguir: 0.0007
  internas: 0.0007
  descendencia: 0.0007
  brújula: 0.0007
  poco: 0.0007
Verosimilitud negativa para 'los': 7.6171



In [120]:
import torch
import torch.nn as nn
import torch.nn.functional as F

# Definir una función para generar una frase con el modelo simple
def generate_sentence(start_word, model, length=5):
    current_word = start_word
    sentence = [current_word]
    
    for _ in range(length):
        idx = word_to_idx.get(current_word, 0)  # Obtener el índice de la palabra actual
        input_tensor = torch.tensor([idx], dtype=torch.long)
        input_one_hot = F.one_hot(input_tensor, num_classes=vocab_size).float()
        
        output = model(input_one_hot)  # Obtener la predicción del modelo
        top_word_idx = torch.argmax(output, dim=1).item()  # Índice de la palabra más probable
        current_word = idx_to_word.get(top_word_idx, '<UNK>')  # Obtener la palabra correspondiente
        
        sentence.append(current_word)  # Añadir la palabra a la oración
    
    return ' '.join(sentence)

# Función para calcular la verosimilitud negativa de una frase
def calculate_negative_likelihood(sentence, model):
    test_indices = [word_to_idx[word] for word in sentence]
    test_tensor = torch.tensor(test_indices[:-1], dtype=torch.long)  # Excluir la última palabra
    test_target = torch.tensor(test_indices[1:], dtype=torch.long)   # Excluir la primera palabra
    
    test_one_hot = F.one_hot(test_tensor, num_classes=vocab_size).float()
    outputs = model(test_one_hot)
    
    criterion = nn.CrossEntropyLoss()
    loss = criterion(outputs, test_target)
    
    return loss.item()

# Generar frases con el modelo simple
print(generate_sentence('macondo', model))
print(generate_sentence('soledad', model))
print(generate_sentence('los', model))

# Calcular la verosimilitud negativa de las frases generadas
generated_sentence_1 = generate_sentence('macondo', model).split()
generated_sentence_2 = generate_sentence('soledad', model).split()
generated_sentence_3 = generate_sentence('los', model).split()

print(f"Verosimilitud negativa para la oración '{' '.join(generated_sentence_1)}': {calculate_negative_likelihood(generated_sentence_1, model):.4f}")
print(f"Verosimilitud negativa para la oración '{' '.join(generated_sentence_2)}': {calculate_negative_likelihood(generated_sentence_2, model):.4f}")
print(f"Verosimilitud negativa para la oración '{' '.join(generated_sentence_3)}': {calculate_negative_likelihood(generated_sentence_3, model):.4f}")


macondo magisterio adiestrarlos permaneció condujo abejas
soledad seguir seguir seguir seguir adiestrarlos
los magisterio adiestrarlos arrastrado magisterio seguir
Verosimilitud negativa para la oración 'macondo adiestrarlos abejas adiestrarlos permaneció permaneció': 7.6170
Verosimilitud negativa para la oración 'soledad macondo seguir adiestrarlos adiestrarlos abejas': 7.6171
Verosimilitud negativa para la oración 'los adiestrarlos seguir seguir permaneció permaneció': 7.6170


## **Conclusión**

Después de realizar múltiples pruebas variando la cantidad de capas, el número de épocas y la tasa de aprendizaje, no se observó una mejora significativa en las frases generadas en comparación con el modelo más simple. Esto sugiere que podrían ser necesarias otras técnicas de procesamiento de datos o diferentes tipos de modelos para mejorar el rendimiento. Además, podría ser crucial contar con un conjunto de datos mucho más grande y desarrollar un manejo más detallado de las relaciones entre palabras y frases para obtener resultados más efectivos.